In [1]:
import random
import pandas as pd
from datetime import datetime

# Temporary workflow
# (1) Download csv with `heroku run make exportdb && make importdb`
# (2) save links to ~/Documents/links.txt in onetab format
# (3) run this script (or otherwise modify the CSV as desired)
# (4) check date cell
# (5) Upload back up with `make csv_to_s3 && heroku run make importdb`
# (6) run `make pipeline` to clean DB
links = pd.read_csv('data/export.csv')
links

/var/folders/3q/_qq6n78n6m92_765npwz342r0000gn/T/ipykernel_82613/564014651.py:12: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  links = pd.read_csv('data/export.csv')


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104386,167383,https://chinai.substack.com/p/chinai-236-the-l...,ChinAI #236: The LLM Implementation Gap,\N,\N,2023-09-12 11:48:48.872995+00,2023-09-12 11:48:48.872995+00,NaN,\N,ChinAI,89.0,0.0,\N
104387,167384,https://www.centreforeffectivealtruism.org/blo...,CEA’s 2017 review and 2018 plans,\N,\N,2023-09-12 11:48:48.874168+00,2023-09-12 11:48:48.874168+00,NaN,\N,CEA,80.0,0.0,\N
104388,167385,https://www.astralcodexten.com/p/highlights-fr...,"Highlights From The Comments On Last Week""s Mo...",\N,\N,2023-09-12 11:48:48.875349+00,2023-09-12 11:48:48.875349+00,NaN,\N,SSC,49.0,0.0,\N
104389,167386,https://www.noahpinion.blog/p/the-next-phase-o...,The next phase of globalization is going to be...,\N,\N,2023-09-12 11:48:48.876687+00,2023-09-12 11:48:48.876687+00,NaN,\N,Noah,80.0,0.0,\N


In [2]:
links['added'].max()

'2023-09-12 11:48:48.878012+00'

In [3]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]['id']

1299        1306
1300        1307
1301        1308
1302        1309
1303        1310
           ...  
104221    167183
104222    167184
104223    167185
104224    167194
104225    167207
Name: id, Length: 13155, dtype: int64

In [4]:
links[links['url'].duplicated()]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
104387,167384,https://www.centreforeffectivealtruism.org/blo...,CEA’s 2017 review and 2018 plans,\N,\N,2023-09-12 11:48:48.874168+00,2023-09-12 11:48:48.874168+00,NaN,\N,CEA,80.0,0.0,\N


In [5]:
links[links['url'].duplicated()]['id']

104387    167384
Name: id, dtype: int64

In [6]:
link_file = open('/Users/peterhurford/Documents/links.txt', 'r')
new_links = link_file.readlines()

In [7]:
def new_entry(links, url, title):
    return {'id': links['id'].max() + 1,
            'url': str(url).strip().replace(',', ''),
            'title': str(title).strip().replace(',', ''),
            'summary': '\\N',
            'domain': '\\N',
            'added': str(datetime.now()),
            'modified': str(datetime.now()),
            'liked': '\\N',
            'category': '\\N',
            'aggregator': 'Custom',
            'seed': random.randint(1, 100),
            'starred': '\\N',
            'tweet': 0}

for i, new_link in enumerate(new_links):
    if i % 10 == 0 or i == len(new_links) - 1:
        print('...{}/{}'.format(i + 1, len(new_links)))
    if ' | ' in new_link:
        new_link = new_link.replace('    - ', '').replace('\n', '')
        entry = new_entry(links, new_link.split(' | ')[0], new_link.split(' | ')[1])
        links = pd.concat([links, pd.DataFrame([entry])], ignore_index=True)

links

...1/114
...11/114
...21/114
...31/114
...41/114
...51/114
...61/114
...71/114
...81/114
...91/114
...101/114
...111/114
...114/114


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104500,167497,https://docs.google.com/document/d/1fAUpMb7Adg...,Lennart's quick unfiltered takes on the lab go...,\N,\N,2023-09-12 18:53:26.150694,2023-09-12 18:53:26.150699,\N,\N,Custom,92.0,0.0,\N
104501,167498,https://www.linkedin.com/posts/philipp-hacker-...,(46) Post,\N,\N,2023-09-12 18:53:26.156337,2023-09-12 18:53:26.156342,\N,\N,Custom,72.0,0.0,\N
104502,167499,https://docs.google.com/document/d/1TgqdVpdVJx...,[TAIGA] 2023-07-17 Offline Licensing - Google ...,\N,\N,2023-09-12 18:53:26.163430,2023-09-12 18:53:26.163436,\N,\N,Custom,86.0,0.0,\N
104503,167500,https://docs.google.com/document/d/1nmZI1EMMMF...,Relative national power over AI development: w...,\N,\N,2023-09-12 18:53:26.170240,2023-09-12 18:53:26.170246,\N,\N,Custom,96.0,0.0,\N


In [8]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
1299,1306,http://blog.beeminder.com/trust/,Trusting Your Divided Self | Beeminder Blog,\N,blog.beeminder.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,10.0,0.0,0
1300,1307,http://powlyglot.com/fi3m-review-part-1-focus/,"""What I Learned from Fluent in 3 Months\, Part...",\N,powlyglot.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,9.0,0.0,0
1301,1308,http://effective-altruism.com/ea/u8/ea_intervi...,"""EA Interview Series\, February 2016: John Sal...",\N,effective-altruism.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,effectivealtruism,Custom,85.0,0.0,0
1302,1309,https://fivethirtyeight.com/features/most-2020...,Most 2020 Candidates Have Something In Common:...,\N,fivethirtyeight.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,1.0,politicalscience,Custom,82.0,0.0,0
1303,1310,http://www.smartpassiveincome.com/backlinking-...,THE Backlinking Strategy That Works – 2014 and...,\N,www.smartpassiveincome.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,entrepreneurship,Custom,59.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104500,167497,https://docs.google.com/document/d/1fAUpMb7Adg...,Lennart's quick unfiltered takes on the lab go...,\N,\N,2023-09-12 18:53:26.150694,2023-09-12 18:53:26.150699,\N,\N,Custom,92.0,0.0,\N
104501,167498,https://www.linkedin.com/posts/philipp-hacker-...,(46) Post,\N,\N,2023-09-12 18:53:26.156337,2023-09-12 18:53:26.156342,\N,\N,Custom,72.0,0.0,\N
104502,167499,https://docs.google.com/document/d/1TgqdVpdVJx...,[TAIGA] 2023-07-17 Offline Licensing - Google ...,\N,\N,2023-09-12 18:53:26.163430,2023-09-12 18:53:26.163436,\N,\N,Custom,86.0,0.0,\N
104503,167500,https://docs.google.com/document/d/1nmZI1EMMMF...,Relative national power over AI development: w...,\N,\N,2023-09-12 18:53:26.170240,2023-09-12 18:53:26.170246,\N,\N,Custom,96.0,0.0,\N


In [9]:
def clean(txt):
    if not isinstance(txt, str):
        return txt
    txt = txt.replace('&nbsp;', '')
    txt = txt.replace('&ldquo;', '"')
    txt = txt.replace('&rdquo;', '"')
    txt = txt.replace('&lsquo;', '\'')
    txt = txt.replace('&rsquo;', '\'')
    txt = txt.replace('&mdash;', '-')
    txt = txt.replace('&ndash;', '-')
    for i in range(10):
        txt = txt.replace('"""', '"')
        txt = txt.replace('\'\'\'', '\'')
    txt = txt.replace('n"t', 'n\'t')
    txt = txt.replace('n""t', 'n\'t')
    return txt

links = links[links['id'].notnull()]   # Drop empty column
links['id'] = links['id'].astype(int)  # Fix float ID issue
links['seed'] = links['seed'].astype(int)
links['tweet'] = links['tweet'].apply(lambda x: 0 if str(x) == '\\N' else str(x).split('.')[0]).astype(int)
links['summary'] = links['summary'].apply(clean)
links['title'] = links['title'].apply(clean)
links = links.sort_values('id')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17,0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59,0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56,0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29,0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104500,167497,https://docs.google.com/document/d/1fAUpMb7Adg...,Lennart's quick unfiltered takes on the lab go...,\N,\N,2023-09-12 18:53:26.150694,2023-09-12 18:53:26.150699,\N,\N,Custom,92,0,\N
104501,167498,https://www.linkedin.com/posts/philipp-hacker-...,(46) Post,\N,\N,2023-09-12 18:53:26.156337,2023-09-12 18:53:26.156342,\N,\N,Custom,72,0,\N
104502,167499,https://docs.google.com/document/d/1TgqdVpdVJx...,[TAIGA] 2023-07-17 Offline Licensing - Google ...,\N,\N,2023-09-12 18:53:26.163430,2023-09-12 18:53:26.163436,\N,\N,Custom,86,0,\N
104503,167500,https://docs.google.com/document/d/1nmZI1EMMMF...,Relative national power over AI development: w...,\N,\N,2023-09-12 18:53:26.170240,2023-09-12 18:53:26.170246,\N,\N,Custom,96,0,\N


In [10]:
links.to_csv('data/export.csv', index=False)